In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from typing import List, Dict, Any, TypedDict, Annotated
from pydantic import BaseModel
from langgraph.constants import Send
import json
from operator import add
from fcgb.cfg.precompiled import get_llm, get_checkpointer
from fcgb.prompt_manager import PromptManager
from rag_eval import RandomQueriesPaperSearchGraph, ChunkEvalGraph, run_all_queries_async
import os

## download documents

In [ ]:
query_runner = RandomQueriesPaperSearchGraph(
    llm=get_llm(llm_model='google'),
    prompts_config={
        'path': '../prompts',
        'random_queries': 'random_queries',
        'paper_queries': 'paper_queries'
    },
    docs_path='../data/docs',
    docs_metadata_path='../data/docs_metadata',
    main_queries_num=3,
    paper_queries_num=10,
    max_results=5,
)

In [30]:
query_runner.run()

Generating random queries for paper search...
Searching for papers with query: explainable AI methods for fraud detection in banking
Searching for papers with query: federated learning privacy preserving techniques healthcare
Searching for papers with query: transformer networks time series forecasting stock market
  and CNNs for Stock Return Predictions
  Conversations between MINT and G7 Countries
  Medical Image Classification
  Healthcare
  Federated Learning and Differential Privacy
  Applications
  IoT based Healthcare Applications A Survey
  Mask Learning
  Maximization
  algorithms using Bfloat 16 in credit card fraud detection
  Detection
Generating queries for paper: A novel approach to increase scalability while training machine learning
  algorithms using Bfloat 16 in credit card fraud detection
Generating queries for paper: SEFraud: Graph-based Self-Explainable Fraud Detection via Interpretative
  Mask Learning
Generating queries for paper: Locally Interpretable One-Class 

{'queries': QueriesModel(queries=['explainable AI methods for fraud detection in banking', 'federated learning privacy preserving techniques healthcare', 'transformer networks time series forecasting stock market']),
 'metadata_files': ['../docs_metadata/2406.11389v1.json',
  '../docs_metadata/1811.08212v1.json',
  '../docs_metadata/2206.12415v1.json',
  '../docs_metadata/2409.13406v1.json',
  '../docs_metadata/2108.02501v3.json',
  '../docs_metadata/2306.17794v1.json',
  '../docs_metadata/2009.08294v1.json',
  '../docs_metadata/2406.15962v1.json',
  '../docs_metadata/2110.03478v2.json',
  '../docs_metadata/2406.05517v1.json',
  '../docs_metadata/2312.15235v1.json',
  '../docs_metadata/2310.07427v3.json',
  '../docs_metadata/2408.12408v1.json',
  '../docs_metadata/2502.09625v1.json',
  '../docs_metadata/2506.01945v1.json']}

## Chunks labeling

In [287]:
memory = get_checkpointer(checkpointer_mode='local', mode='test')

In [ ]:
chunk_eval_graph = ChunkEvalGraph(
    llm=get_llm(llm_model='google'),
    prompts_config={
        'path': '../prompts',
        'chunk_eval_system': 'chunk_eval_system',
        'chunk_eval_task': 'chunk_eval_task',
        'doc_context_system': 'doc_context_system',
        'doc_context_update': 'doc_context_update',
        'doc_context_aggregation': 'doc_context_aggregation'
    },
    memory=memory,
    docs_metadata_path='../data/docs_metadata',
    saving_path='../data/chunk_eval_results',
    chunk_size=600,
    chunk_overlap=0,
    context_agg_interval=5,
    prompt_manager_spec={}
)

In [289]:
await run_all_queries_async(chunk_eval_graph, "../docs_metadata/2502.09625v1.json")

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=3]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=4]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=2]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=5]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=0]

...

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=7]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=1]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=8]

Evaluating Chunks:   0%|          | 0/59 [00:00<?, ?it/s, Negative=0, Positive=0, Query_id=9]

Saving evaluation results for query 8 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 1 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 4 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 2 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 0 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 5 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 6 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 7 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 3 on document ../docs/2502.09625v1.pdf...
Saving evaluation results for query 9 on document ../docs/2502.09625v1.pdf...


In [ ]:
docs_metadata_path = '../data/docs_metadata'

metadata_file = '2502.09625v1.json'
query_idx = 1

with open(os.path.join(docs_metadata_path, metadata_file), 'r') as f:
    metadata = json.load(f)

paper_query = metadata['queries'][query_idx]

inputs = {
    'query': paper_query,
    'query_idx': query_idx,
    'doc_file': metadata['path'],
    'title': metadata['title'],
    'summary': metadata['summary']
}

inputs

{'query': 'How does Stockformer differ from the original Transformer model?',
 'query_idx': 1,
 'doc_file': '../docs/2502.09625v1.pdf',
 'title': 'Transformer Based Time-Series Forecasting for Stock',
 'summary': 'To the naked eye, stock prices are considered chaotic, dynamic, and\nunpredictable. Indeed, it is one of the most difficult forecasting tasks that\nhundreds of millions of retail traders and professional traders around the\nworld try to do every second even before the market opens. With recent advances\nin the development of machine learning and the amount of data the market\ngenerated over years, applying machine learning techniques such as deep\nlearning neural networks is unavoidable. In this work, we modeled the task as a\nmultivariate forecasting problem, instead of a naive autoregression problem.\nThe multivariate analysis is done using the attention mechanism via applying a\nmutated version of the Transformer, "Stockformer", which we created.'}

In [251]:
thread_id = 'Stockformer1'

In [252]:
state = chunk_eval_graph.run_with_progress(**inputs, thread_id=thread_id)

Chunking document: ../docs/2502.09625v1.pdf into 59 chunks.


Evaluating Chunks: 100%|██████████| 59/59 [02:26<00:00,  2.49s/it, Negative=48, Positive=11]         

Saving evaluation results for query 1 on document ../docs/2502.09625v1.pdf...


In [253]:
state = memory.get({'configurable': {'thread_id': thread_id}})['channel_values']

In [254]:
positive_evaluations = sum([1 for chunk in state['chunks_eval'] if chunk['label']])
negative_evaluations = sum([1 for chunk in state['chunks_eval'] if not chunk['label']])
print(f"Positive evaluations: {positive_evaluations}\nNegative evaluations: {negative_evaluations}")

print("\nCONTEXT:")
for line in state['context'].split('\n'):
    print(line)

Positive evaluations: 11
Negative evaluations: 48

CONTEXT:
```
## Summary of previous sections:
- Stockformer is a modified Transformer model for hourly stock price forecasting, utilizing ProbSparse Attention and Self-attention Distilling for efficiency.
- Stockformer's performance benefits from larger embedding sizes and more attention heads.
- Stockformer with ProbSparse Attention is more efficient than Full Attention and more profitable than LSTM models.
- Future research will focus on incorporating more stock tickers, dynamic training methods, and exploring Time2Vec and TimeFrame2Vec temporal encoding to improve the model's profitability and adaptability.
- The paper provides a list of references used in the research.

## Current Section:
The paper provides a list of references used in the research. This section lists academic papers and publications that were cited in the study. The list of references continues from the previous chunk, completing the bibliography.
```


In [255]:
for chunk in state['chunks_eval']:
    if chunk['label']:
        print(f"\nChunk {chunk['idx']} (Positive):")
        for line in chunk['text'].split('\n'):
            print(line)
        print("\n---\n")


Chunk 1 (Positive):
the market generated over years, applying machine learning
techniques such as deep learning neural networks is unavoidable.
In this work, we modeled the task as a multivariate forecast-
ing problem, instead of a naive autoregression problem. The
multivariate analysis is done using the attention mechanism via
applying a mutated version of the Transformer, ”Stockformer”,
which we created.
I. I NTRODUCTION
Predicting the financial time series such as stock price
means predicting the behavior of the stock price steps ahead of
the series with the help of various variables. By knowing the

---


Chunk 14 (Positive):
In this project, we implemented Stockformer on the top of
the Transformer, discussed issues of naive Transformer, and
changed the original architecture to fit with the financial ticker
forecasting task.
III. P ROBLEM FORMULATION
Although the goal for the neural network is to predict the
stock price ahead, the task for this project is to assist traders
to make

In [171]:
thread_id = 'Byzantine7'

In [172]:
state = chunk_eval_graph.graph.invoke(
    input=inputs,
    config={'configurable': {'thread_id': thread_id}},
    interrupt_before='chunk_eval'
)

Chunking document: ../docs/2009.08294v1.pdf into 44 chunks.


In [182]:
state = chunk_eval_graph.graph.invoke(
    input=None,
    config={'configurable': {'thread_id': thread_id}},
    interrupt_before='chunk_eval'
)

positive_evaluations = sum([1 for chunk in state['chunks_eval'] if chunk['label']])
negative_evaluations = sum([1 for chunk in state['chunks_eval'] if not chunk['label']])
print(f"Positive evaluations: {positive_evaluations}\nNegative evaluations: {negative_evaluations}")

print("\nCONTEXT:")
for line in state['context'].split('\n'):
    print(line)

Positive evaluations: 3
Negative evaluations: 7

CONTEXT:
## Summary of previous sections:
- The paper examines robust aggregation methods in federated learning (FL) for healthcare, aiming to protect privacy and defend against malicious clients by detecting and discarding them during training.
- Experiments use healthcare datasets to evaluate robust FL aggregation against poisoning attacks, demonstrating that privacy-preserving methods can be combined with Byzantine-robust aggregation without significantly impacting learning.
- The paper addresses the challenges of sharing private patient data and introduces federated learning (FL) as a solution, while also acknowledging privacy and robustness concerns like biased datasets and poisoning attacks.
- The paper's approach involves using k-anonymity and differential privacy (DP), modeling poisoning attack strategies, and using healthcare datasets to evaluate robust aggregation methods in FL.
- The main contributions include evaluating robus